<a href="https://colab.research.google.com/github/Nathan2605/k-nearest-neighbors-project-tutorial/blob/main/Proyecto_4geeks_KNN_NH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

####DATOS

In [4]:
#url
url_movies = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv'
url_credits = 'https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv'

#cargar bases de datos
df_m = pd.read_csv(url_movies)
df_c = pd.read_csv(url_credits)

#pegar bases de datos
df = df_m.merge(df_c, left_on='title', right_on='title')

#apartar columnas de interes
df = df[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [5]:
##funciones para limpiar

#sacar nombres en genres y keywords
def clean(data):
  try:
    data = json.loads(data)
    elements = []
    for i in range(0,len(data)):
      elements.append(str(data[i]['name']))
    return elements
  except (TypeError, json.JSONDecodeError):
    return []

#sacar 3 primeros de cast
def clean2(data):
  try:
    data = json.loads(data)
    elements = []
    for i in range(0,min(3, len(data))):
      elements.append(str(data[i]['name']))
    return elements
  except (TypeError, json.JSONDecodeError):
    return []

#sacar nombre directo en crew
def clean3(data):
  try:
    data = json.loads(data)
    for i in range(len(data)):
      if data[i]['job'] == 'Director':
        return str(data[i]['name'])
  except (TypeError, json.JSONDecodeError):
    return []

#convertir en lista a overview
def clean4(data):
  if type(data) != list:
    return [str(data)]
  else:
    return []

In [6]:
#limpiar
df['genres'] = df['genres'].apply(clean)
df['keywords'] = df['keywords'].apply(clean)
df['cast'] = df['cast'].apply(clean2)
df['crew'] = df['crew'].apply(clean3)
df['overview'] = df['overview'].apply(clean4)

# Convertir a string
for col in df.columns:
  if col != 'movie_id':
    df[col] = df[col].astype(str)

#columna tags
df['tags'] = df['overview'] + df['genres'] + df['keywords'] + df['cast'] + df['crew']
df['tags'] = df['tags'].apply(lambda x: "".join(x))
df['tags'] = df['tags'].replace(',', ' ', regex=True)

####KNN

In [10]:
#vectorizar tags
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["tags"])

In [16]:
model = NearestNeighbors(n_neighbors=6)
model.fit(tfidf_matrix)

NearestNeighbors(n_neighbors=6)

In [17]:
def get_movie_recommendations(movie_title):
    movie_index = df[df["title"] == movie_title].index[0]
    distances, indices = model.kneighbors(tfidf_matrix[movie_index])
    similar_movies = [(df["title"][i], distances[0][j]) for j, i in enumerate(indices[0])]
    return similar_movies[1:]

input_movie = "How to Train Your Dragon"
recommendations = get_movie_recommendations(input_movie)
print("Film recommendations '{}'".format(input_movie))
for movie, distance in recommendations:
    print("- Film: {}".format(movie))

Film recommendations 'How to Train Your Dragon'
- Film: How to Train Your Dragon 2
- Film: Dragon Nest: Warriors' Dawn
- Film: Pete's Dragon
- Film: George and the Dragon
- Film: Eragon
